## Homework

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

In [1]:
import warnings

import numpy as np
import pandas as pd

In [2]:
!mkdir dataset

In [4]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://yadi.sk/d/guqki4UI4hFlXQ'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('dataset/data.zip', 'wb') as f:   # Здесь укажите нужный путь к файлу
    f.write(download_response.content)

In [5]:
import zipfile

with zipfile.ZipFile("dataset/data.zip", mode="r") as archive:
    archive.extractall("dataset")

In [6]:
import zipfile

with zipfile.ZipFile("dataset/hidden_task.zip", mode="r") as archive:
    archive.extractall("dataset")

In [7]:
!del "dataset\hidden_task.zip"
!del "dataset\open_task.zip"
!del "dataset\data.zip"

In [18]:
# считываем данные
qid_query = pd.read_csv("dataset/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("dataset/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("dataset/hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [19]:
qid_url_rating_hostid.head()

,qid,url,rating,hostid
0,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07,7
1,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07,7
2,402111,http://802351.info/5964-v-avstralii.html,0.00,13
3,402111,http://auscommunity.com/blog/jobs/,0.00,53
4,402111,http://auscommunity.com/tag/%D1%84%D0%BE%D1%82...,0.00,53


In [20]:
def mean_PFoundK(df: pd.DataFrame, K: int = 10, p_break: float = 0.15) -> float:
    # Calculate average PFound@K
    qid_hostid_max_rel = df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    qid_top_k_hostid = qid_hostid_max_rel.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)
    # Порядковые номера для hostid внутри каждого qid (0: k - 1)
    qid_top_k_hostid['n'] = qid_top_k_hostid.groupby('qid').cumcount()

    # Рассчитаем вспомогательные множители для расчета plook: в формуле это (1 - Rel) и (1 - pBreak)
    qid_top_k_hostid['one_minus_rating'] = (1 - qid_top_k_hostid['rating']).shift(1)
    qid_top_k_hostid['one_minus_p_break'] = 1 - p_break
    qid_top_k_hostid.loc[qid_top_k_hostid['n'] == 0, ['one_minus_p_break', 'one_minus_rating']] = 1

    # Расчитаем вспомогательную колонку, из которой затем просто используя cumprod получим plool
    qid_top_k_hostid['multiplier'] = qid_top_k_hostid['one_minus_rating'] * qid_top_k_hostid['one_minus_p_break']

    qid_top_k_hostid['plook'] = qid_top_k_hostid.groupby('qid')['multiplier'].cumprod()
    qid_top_k_hostid['pfound'] = qid_top_k_hostid['plook'] * qid_top_k_hostid['rating']
    
    pfound_qid = qid_top_k_hostid.groupby('qid')['pfound'].sum()
    return pfound_qid.mean()

In [21]:
avg_pfound = mean_PFoundK(qid_url_rating_hostid)
print(f'Средний PFound по всем query: {avg_pfound}')

Средний PFound по всем query: 0.5822199638393889


## Решение с сайта

In [22]:
def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound

qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound
qid_query[qid_query["qid"] == qid_max]

,qid,query
14,692308,бесплатный просмотр камеди клаб


In [23]:
avg_pfound_original = qid_pfound.mean()
print(f'Средний pFound по всем query (расчет с сайта): {avg_pfound_original}')

Средний pFound по всем query (расчет с сайта): 0.5822199638393889


In [24]:
assert np.abs(avg_pfound - avg_pfound_original) < 1e-6, 'PFound metrics is not equal'